In [12]:
# Standard library
import json
import math
import os
import time
from collections import deque
from pathlib import Path
from typing import Dict, Any, List

# Data manipulation and analysis
import pandas as pd
import numpy as np
import geopandas as gpd

# Database
import duckdb

# Network analysis
import networkx as nx
import osmnx as ox

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import pydeck as pdk

# LangChain and LLM
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

# Rich console output
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
from rich import box

# Jupyter display
from IPython.display import display, Markdown as IPMarkdown, HTML, clear_output

# Configure OSMnx
ox.settings.use_cache = True
ox.settings.log_console = True

# Initialize console for Rich output
console = Console()

In [13]:
print("🌍 DreamStreets Data Loader")
print("=" * 50)

# Use the center point of Chinatown, Manhattan
# Approximate coordinates for Chinatown center (Canal & Mott Street area)
chinatown_coords = (40.7168, -73.9986)
dist = 500  # meters (adjust as needed for coverage)

print(f"📍 Loading street network for: Chinatown, Manhattan")
print(f"   Using coordinates: {chinatown_coords}")
print(f"   Radius: {dist} meters")

# Download street network using point and distance
G_chinatown = ox.graph_from_point(chinatown_coords, dist=dist, network_type='drive')

# Convert node IDs to strings (required for analysis tools)
G_chinatown = nx.relabel_nodes(G_chinatown, {n: str(n) for n in G_chinatown.nodes()})

print(f"✅ Loaded {G_chinatown.number_of_nodes()} nodes, {G_chinatown.number_of_edges()} edges")

# Save to GraphML
ox.save_graphml(G_chinatown, "chinatown.graphml")
print(f"💾 Saved to chinatown.graphml")

🌍 DreamStreets Data Loader
📍 Loading street network for: Chinatown, Manhattan
   Using coordinates: (40.7168, -73.9986)
   Radius: 500 meters
✅ Loaded 139 nodes, 274 edges
💾 Saved to chinatown.graphml


In [14]:
import osmnx as ox
import pandas as pd

# Use the center point of Chinatown, Manhattan
chinatown_coords = (40.7168, -73.9986)
dist = 1000  # meters

print(f"🏢 Loading POI data for: Chinatown, Manhattan")
print(f"   Using coordinates: {chinatown_coords}")
print(f"   Radius: {dist} meters")

# Define tags to download
tags = {
    'amenity': True,
    'building': True,
    'healthcare': True,
    'shop': True,
    'tourism': True,
    'emergency': True
}

# Download POIs using point and distance
pois_chinatown = ox.features_from_point(chinatown_coords, tags, dist=dist)

# Extract coordinates (centroids for polygons)
pois_chinatown['lon'] = pois_chinatown.geometry.centroid.x
pois_chinatown['lat'] = pois_chinatown.geometry.centroid.y

# Convert to regular dataframe for DuckDB
pois_chinatown_df = pd.DataFrame(pois_chinatown.drop(columns='geometry'))

print(f"✅ Loaded {len(pois_chinatown_df)} POIs")
if 'amenity' in pois_chinatown_df.columns:
    print(f"   Top amenities: {pois_chinatown_df['amenity'].value_counts().head(3).to_dict()}")

🏢 Loading POI data for: Chinatown, Manhattan
   Using coordinates: (40.7168, -73.9986)
   Radius: 1000 meters
✅ Loaded 10611 POIs
   Top amenities: {'bench': 1317, 'restaurant': 563, 'waste_basket': 547}


/tmp/ipykernel_2164/3020053195.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_chinatown['lon'] = pois_chinatown.geometry.centroid.x
/tmp/ipykernel_2164/3020053195.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_chinatown['lat'] = pois_chinatown.geometry.centroid.y


In [15]:
# Create DuckDB database with spatial extensions
db_path = "chinatown.duckdb"
print(f"🗄️ Creating spatial database: {db_path}")

# Connect to DuckDB
conn_chinatown = duckdb.connect(db_path)

# Install and load spatial extension
conn_chinatown.execute("INSTALL spatial;")
conn_chinatown.execute("LOAD spatial;")

# Prepare nodes data from graph
nodes_data = []
for node, data in G_chinatown.nodes(data=True):
    nodes_data.append({
        'node_id': str(node),
        'lat': data.get('y', 0),
        'lon': data.get('x', 0),
        'street_count': data.get('street_count', 0)
    })

nodes_df = pd.DataFrame(nodes_data)

# Create nodes table
conn_chinatown.execute("""
    CREATE OR REPLACE TABLE nodes AS 
    SELECT *, ST_Point(lon, lat) as geom 
    FROM nodes_df
""")

# Create POIs table
conn_chinatown.execute("""
    CREATE OR REPLACE TABLE pois AS 
    SELECT *, ST_Point(lon, lat) as geom 
    FROM pois_chinatown_df
""")

# Create spatial indexes
conn_chinatown.execute("CREATE INDEX idx_nodes_geom ON nodes USING RTREE(geom)")
conn_chinatown.execute("CREATE INDEX idx_pois_geom ON pois USING RTREE(geom)")

print(f"✅ Database created with {len(nodes_df)} nodes and {len(pois_chinatown_df)} POIs")

🗄️ Creating spatial database: chinatown.duckdb
✅ Database created with 139 nodes and 10611 POIs


In [16]:
# Load street network for Cox's Bazar, Bangladesh
cox_bazar_coords = (21.2143, 92.1666)
cox_bazar_dist = 500  # meters

print(f"📍 Loading street network for: Cox's Bazar, Bangladesh")

# Download street network from coordinates
G_coxbazar = ox.graph_from_point(cox_bazar_coords, dist=cox_bazar_dist, network_type='drive')

# Convert node IDs to strings
G_coxbazar = nx.relabel_nodes(G_coxbazar, {n: str(n) for n in G_coxbazar.nodes()})

print(f"✅ Loaded {G_coxbazar.number_of_nodes()} nodes, {G_coxbazar.number_of_edges()} edges")

# Save to GraphML
ox.save_graphml(G_coxbazar, "coxbazar.graphml")
print(f"💾 Saved to coxbazar.graphml")

📍 Loading street network for: Cox's Bazar, Bangladesh
✅ Loaded 31 nodes, 66 edges
💾 Saved to coxbazar.graphml


In [17]:
# Load POI data for Cox's Bazar
print(f"🏢 Loading POI data for: Cox's Bazar")

# Download POIs from OpenStreetMap
pois_coxbazar = ox.features_from_point(cox_bazar_coords, tags, dist=cox_bazar_dist)

# Extract coordinates
pois_coxbazar['lon'] = pois_coxbazar.geometry.centroid.x
pois_coxbazar['lat'] = pois_coxbazar.geometry.centroid.y

# Convert to dataframe
pois_coxbazar_df = pd.DataFrame(pois_coxbazar.drop(columns='geometry'))

print(f"✅ Loaded {len(pois_coxbazar_df)} POIs")
print(f"   Top amenities: {pois_coxbazar_df['amenity'].value_counts().head(3).to_dict()}")

🏢 Loading POI data for: Cox's Bazar
✅ Loaded 3984 POIs
   Top amenities: {'hospital': 2, 'refugee_site': 2, 'doctors': 1}


/tmp/ipykernel_2164/1647863537.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_coxbazar['lon'] = pois_coxbazar.geometry.centroid.x
/tmp/ipykernel_2164/1647863537.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois_coxbazar['lat'] = pois_coxbazar.geometry.centroid.y


In [18]:
# Create Cox's Bazar database
db_path_cox = "coxbazar.duckdb"
print(f"🗄️ Creating spatial database: {db_path_cox}")

conn_coxbazar = duckdb.connect(db_path_cox)

# Install spatial extension
conn_coxbazar.execute("INSTALL spatial;")
conn_coxbazar.execute("LOAD spatial;")

# Prepare nodes data
cox_nodes_data = []
for node, data in G_coxbazar.nodes(data=True):
    cox_nodes_data.append({
        'node_id': str(node),
        'lat': data.get('y', 0),
        'lon': data.get('x', 0),
        'street_count': data.get('street_count', 0)
    })

cox_nodes_df = pd.DataFrame(cox_nodes_data)

# Create tables
conn_coxbazar.execute("""
    CREATE OR REPLACE TABLE nodes AS 
    SELECT *, ST_Point(lon, lat) as geom 
    FROM cox_nodes_df
""")

conn_coxbazar.execute("""
    CREATE OR REPLACE TABLE pois AS 
    SELECT *, ST_Point(lon, lat) as geom 
    FROM pois_coxbazar_df
""")

# Create indexes
conn_coxbazar.execute("CREATE INDEX idx_nodes_geom ON nodes USING RTREE(geom)")
conn_coxbazar.execute("CREATE INDEX idx_pois_geom ON pois USING RTREE(geom)")

print(f"✅ Database created with {len(cox_nodes_df)} nodes and {len(pois_coxbazar_df)} POIs")

🗄️ Creating spatial database: coxbazar.duckdb
✅ Database created with 31 nodes and 3984 POIs


In [21]:
conn_chinatown.close()
conn_coxbazar.close()